# Cria Tabelas

## Bibliotecas

In [1]:
from selenium import webdriver
import requests
import time
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns; sns.set()
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
import camelot
import folium
from folium import plugins
import pathlib
import shutil


pd.options.display.float_format = '{:.2f}'.format
home = Path.home()
parent = pathlib.Path().absolute()
cidade='piracicaba'
path_ipca_midlle = Path(parent, "bases_brutas/ipca/ipca2.csv")

path_download2 = Path(parent, "bases_brutas/ipca/ipca.csv")
path_download2_1 = Path(parent, "bases_brutas/ipca/ipca_full.csv")

path_download3 = Path(parent,"bases_brutas/{}/prefeitura/receitas/receita_bimestres_int.csv".format(cidade))

path_download4_1 = Path(parent,"bases/{}/receita_bimestres.csv".format(cidade))
path_download4_2 = Path(parent,"bases/{}/receita_funcao.csv".format(cidade))
path_download4_3 = Path(parent,"bases_brutas/{}/prefeitura/receitas/receita_impostos_temp.csv".format(cidade))
path_download4_3_1 = Path(parent,"bases/{}/receita_impostos.csv".format(cidade))
path_download4_3_2 = Path(parent,"bases/{}/receita_transf.csv".format(cidade))


path_download5 =  Path(parent,"bases/{}/receita_ano_pct.csv".format(cidade))

path_download6 = Path(parent,"bases_brutas/{}/prefeitura/despesas/despesa_bimestres_int.csv".format(cidade))

path_download7 = Path(parent,"bases/{}/despesa_bimestres.csv".format(cidade))

path_download8 = Path(parent,"bases/{}/despesa_ano_pct.csv".format(cidade))

path_download9 = Path(parent,"bases_brutas/{}/prefeitura/despesas/despesa_bimestre_funcao_temp.csv".format(cidade))

path_download10 = Path(parent,"bases/{}/".format(cidade))

path_download11 = Path(parent,"bases_brutas/{}/covid/covid.csv".format(cidade))

path_download12 = Path(parent,"bases_brutas/{}/covid/covid2.csv".format(cidade))

path_download13 = Path(parent,"bases_brutas/{}/eleicao/vereadores/votacao.csv".format(cidade))

path_download14 = Path(parent,"bases_brutas/{}/eleicao/vereadores/sessoes_pira.csv".format(cidade))

path_download15 = Path(parent,"bases/{}/covid.csv".format(cidade))

path_download16 = Path(parent,"bases_brutas/{}/camara/despesas_cam.xlsx".format(cidade))
path_download16_1 = Path(parent,"bases_brutas/{}/camara/despesas_cam1.xlsx".format(cidade))
path_download16_2 = Path(parent,"bases_brutas/{}/camara/despesas_cam2.xlsx".format(cidade))
path_download16_3 = Path(parent,"bases_brutas/{}/camara/despesas_cam3.xlsx".format(cidade))
path_download16_4 = Path(parent,"bases_brutas/{}/camara/despesas_cam4.xlsx".format(cidade))
path_download16_5 = Path(parent,"bases_brutas/{}/camara/despesas_cam5.xlsx".format(cidade))
path_download16_6 = Path(parent,"bases_brutas/{}/camara/despesas_cam6.xlsx".format(cidade))
path_download16_7 = Path(parent,"bases_brutas/{}/camara/despesas_cam7.xlsx".format(cidade))
path_download16_8 = Path(parent,"bases_brutas/{}/camara/despesas_cam8.xlsx".format(cidade))
path_download16_9 = Path(parent,"bases_brutas/{}/camara/despesas_cam9.xlsx".format(cidade))
path_download16_10 = Path(parent,"bases_brutas/{}/camara/despesas_cam10.xlsx".format(cidade))

path_download17 = Path(parent,"bases/{}/despesas_cam.xlsx".format(cidade))
path_download18 = Path(parent,"bases/{}/top100_cam.xlsx".format(cidade))

path_download19 = Path(parent,"bases/{}/vacinas.csv".format(cidade))

path_download20= Path(parent,"bases/{}/20210402_isolamento.csv".format(cidade))

## Funções

In [2]:
# Função retorna o mes escrito:
def mes(x):
    if x == '01':
        return 'fevereiro '
    elif x == '02':
        return 'abril '
    elif x == '03':
        return 'junho '
    elif x == '04':
        return 'agosto '
    elif x == '05':
        return 'outubro '
    else:
        return 'dezembro '

# Função retorna o mes do bimestre:
def mes_ind(x):
    if x == '01':
        return '02'
    elif x == '02':
        return '04' 
    elif x == '03':
        return '06'
    elif x == '04':
        return '08'
    elif x == '05':
        return '10'
    else:
        return '12'
    
# Função que arruma as datas da df para formato timestamp:
def arruma_data(df):
    df['Data'] = df['Bimestre'].apply(mes)
    df['tempo'] = df['Bimestre'].apply(mes_ind)
    df['tempo'] = df['tempo']+df['Ano']
    df['tempo'] = pd.to_datetime(df['tempo'])
    df['Data'] = df['Data'] + df['Ano']
    df.set_index('Data',inplace=True)
    return df
    
# Função que arruma os numeros  do df 'RECEITA' para formato float:
def arruma_num_rec(df):
    for i in ['Previsao Inicial', 'Previsao Atualizada (a)',
       'Receitas Realizadas no Bimestre (b)', '% (b/a)',
       'Receitas Realizadas até o Bimestre (c)', '% (c/a)', 'SALDO (a-c)']:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros  do df 'DESPESAS' para formato float:
def arruma_num_des(df):
    colunas=['DOTAÇÃO INICIAL (d)',
       'DOTAÇÃO ATUALIZADA (e)', 'DESPESAS EMPENHADAS NO BIMESTRE',
       'DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)', 'SALDO (g) =(e-f)',
       'DESPESAS LIQUIDADAS NO BIMESTRE',
       'DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)', 'SALDO (i) = (e-h)',
       'DESPESAS PAGASATÉ O BIMESTRE(j)']
    for i in colunas:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: str(x))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros  do df 'DESPESA-FUNCAO' para formato float:
def arruma_num_des_fun(df):
    colunas=['DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE',
              'DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)',('% (b/total b)'),'SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE',
              'DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','% (d/total d)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
    for i in colunas:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: str(x))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros  do df 'RECEITA-IMPOSTOS' para formato float:
def arruma_num_rec_imp(df):
    colunas=['RECEITAS CORRENTES (I)','Impostos, Taxas e Contribuições de Melhoria','IPTU','ISS','ITBI','IRRF',
             'Outros Impostos, Taxas e Contribuições de Melhoria','Contribuições','Receita Patrimonial',
             'Rendimentos de Aplicação Financeira','Outras Receitas Patrimoniais',
             'Receita de Serviços','Transferências Correntes','Cota-Parte do FPM',
             'Cota-Parte do ICMS','Cota-Parte do IPVA','Cota-Parte do ITR',
             'Transferências da LC nº 61/1989','Transferências do FUNDEB','Outras Transferências Correntes']
    for i in colunas:
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: str(x))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace('.',''))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x:x.replace(',','.'))
        df.loc[:,(i)]= df.loc[:,(i)].apply(lambda x: float(x))
    return df

# Função que arruma os numeros unicos para formato float:
def arruma_numero_unico(num):
    num= num.replace('.','')
    num= num.replace(',','.')
    num= float(num)
    return num

# Função que aplica a inflação no df RECEITA:
def aplica_inflacao_rec(filtered_df):
    for i in [1,2,3,5,7]:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]/filtered_df.iloc[j,11])*100
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que aplica a inflação no df DESPESA:
def aplica_inflacao_des(filtered_df):
    colunas=[1,2,3,4,5,6,7,8,9]
    for i in colunas:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]/filtered_df.iloc[j,13])*100
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que aplica a inflação no df DESPESA-FUNÇÃO:
def aplica_inflacao_des_fun(filtered_df):
    colunas=[1,2,3,4,7,8]
    for i in colunas:
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]*100)/filtered_df.iloc[j,14]
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que aplica a inflação no df DESPESA CÂMARA:
def aplica_inflacao_cam(filtered_df):
    for i in [8,9,10]:
        for j in range(len(filtered_df.iloc[:,0])):
            if filtered_df.iloc[j,11]=='2021':
                continue
            else:
                filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]/filtered_df.iloc[j,12])*100
    return filtered_df

# Função que aplica a inflação no df RECEITA IMPOSTOS:
def aplica_inflacao_rec_imp(filtered_df):
    for i in range(1,20):
        for j in range(len(filtered_df.iloc[:,0])):
            filtered_df.iloc[j,i] = (filtered_df.iloc[j,i]*100)/filtered_df.iloc[j,20]
            filtered_df.iloc[j,i] = filtered_df.iloc[j,i]/1000000  
    return filtered_df

# Função que substitui '-' por 0:
def acha_string(x):
    if x=='-':
        return 0
    else:
        return int(x)

# Função que arruma no df VEREADORES
def numeros_sessoes(x):
    sessoes=[]
    lista_sessoes=x.split(';')
    for i in range(len(lista_sessoes)):
        if len(lista_sessoes[i].split())>=4:
            sessoes.extend(list(range(int(lista_sessoes[i].split()[1].replace('ª','').replace(' ','')),int(lista_sessoes[i].split()[3].replace('ª','').replace('.','').replace(' ',''))+1)))
        else:
            sessoes.append(lista_sessoes[i].replace('ª','').replace(' ',''))
    return sessoes

def retorna_posicao(x):
    
    for i in range(len(locais['Seções2'])):
        if str(x) in locais['Seções2'].iloc[i]:
            return locais['Latitude'].iloc[i]
        if x in locais['Seções2'].iloc[i]:
            return locais['Latitude'].iloc[i]
    return 0


def votos_totais_secao(x):
    for i in range(len(df.Seção)):
        if x == df.Seção.iloc[i]:
            return df.Votos.iloc[i]
    return -1

def modulo(x):
    if x > 0:
        return x
    else:
        return x*(-1)
    
def eh_par(x):
    if x % 2 == 0:
        return 1
    else:
        return 0
    
def descobre_mes(x):
    if x == 1:
        return 'janeiro'
    elif x == 2:
        return 'fevereiro'
    elif x == 3:
        return 'março'
    elif x == 4:
        return 'abril'
    elif x == 5:
        return 'maio'
    elif x == 6:
        return 'junho'
    elif x == 7:
        return 'julho'
    elif x == 8:
        return 'agosto'
    elif x == 9:
        return 'setembro'
    elif x == 10:
        return 'outubro'
    elif x == 11:
        return 'novembro'
    elif x == 12:
        return 'dezembro'
    else:
        return "sei la"



    
def dia_de_ontem(hoje):
    dist = int(hoje.strftime('%d'))-1
    if dist < 10:
        return 'O'+str(dist)
    else:
        return str(dist)
    
def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 10:
        time.sleep(0.2)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.xlsx'):
                dl_wait = True
        seconds += 1
    return seconds

def multiselect_set_selections(driver, element_id, labels):
    el = driver.find_element_by_id(element_id)
    for option in el.find_elements_by_tag_name('option'):
        if option.text in labels:
            option.click()

## IPCA

### Download da base bruta

#### OBS: devemos modificar o arquivo para excel e mudar o nome para ipca2

In [20]:
#*************MODIFICAR O path_download PARA A PASTA DE DOWNLOAD DO SEU PC*************

here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)

cidade='piracicaba'
d = Path(home,'Downloads')
m=Path(parent,"bases_brutas/ipca/")
source_dir = d
middle_dir = m

hoje = datetime.now()
dhoje = hoje.strftime('%d')
mes= hoje.strftime('%m')

arq1='2021{}{}061917.csv'.format(dhoje,mes)
arq2='ipca2.csv'

link = 'https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas'
print(link)
driver.get(link)
driver.implicitly_wait(10)
driver.implicitly_wait(10)
driver.implicitly_wait(30)
multiselect_set_selections(driver,'tabelasidra20171059636838export',['csv'])
driver.quit()

file_names = os.listdir(source_dir)
for file_name in file_names:
    if (file_name[:8]==arq1[:8]):
        shutil.move(os.path.join(source_dir, file_name), middle_dir)
        os.rename(os.path.join(middle_dir, file_name),os.path.join(middle_dir, arq2))
        break


https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas


### Rodar quando já tiver a base instalada

In [21]:
#Abre a base de dados baixada
ipca = pd.read_csv(path_ipca_midlle,';')
ipca=ipca[:2]
ipca.reset_index(inplace=True)
ipca.columns=ipca.iloc[0]
ipca=ipca.iloc[1:].T

#Arruma colunas
ipca.columns = ipca.iloc[0]
ipca = ipca.iloc[1:]
ipca.dropna(axis=0,inplace=True)
ipca = ipca.loc['fevereiro 2016':]
ipca['Brasil']=ipca['Brasil'].astype(float)
ipca['Outro']=ipca['Brasil'].shift(-1)
ipca['Acumulado'] = [0]*len(ipca['Brasil'])
ipca.iloc[-1,0]=100
df1=ipca

i=-1
while i >= -len(df1['Brasil']):
    ant=i+1
    if i==-1:
        df1.iloc[i , 2]=100
    else:
        df1.iloc[i , 2]=(100/(100+df1.iloc[i , 1])*df1.iloc[ant , 2])
    i=i-1

df1.drop(['Brasil','Outro'],axis=1,inplace=True)
df1.columns = ['base = ultimo periodo']
df1.reset_index(inplace=True)
df1.set_index(0,inplace=True)
#Exporta os dados
ipca_full=df1
ipca_full.to_csv(path_download2_1, index = True)

In [23]:
#Arruma as datas para bimestre
meses = ['fevereiro','abril','junho','agosto','outubro','dezembro']
ano = ['2016','2017','2018','2019','2020','2021']
lista = []
for i in ano:
    for j in meses:
        data="{} {}".format(j,i)
        lista.append(data)
df1 = df1[df1.index.isin(lista)]
if len(df1.index)%2:
    pass
else:
    df1 = df1.iloc[:] #ALTERAR AQUI A DATA PARA OUTROS MESES
#Exporta os dados
ipca=df1
ipca.to_csv(path_download2, index = True)
ipca

,base = ultimo periodo
0,
fevereiro 2016,80.91
abril 2016,81.75
junho 2016,82.68
agosto 2016,83.47
outubro 2016,83.76
dezembro 2016,84.16
fevereiro 2017,84.76
abril 2017,85.09
junho 2017,85.15


## Receitas por bimestre

In [24]:
bimestre = ['01','02','03','04','05','06']
ano = ['2021','2020','2019','2018','2017','2016']
receita_orcamentaria=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j)==2021:
            if int(i)>1:
                continue
            else:
                file='http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/2021/declaracaoRREO1oBim2021.pdf'
        if int(j) ==2020:
            if int(i)==6:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/2021-02RREO6Bim-declaracao.pdf'
            else:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        else:
            file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
                
        todo = camelot.read_pdf(file,pages='1-2')
        receita_orcamentaria_temp=todo[1].df.iloc[4:]
        receita_orcamentaria_temp=receita_orcamentaria_temp.append(todo[3].df.iloc[3:])
        receita_orcamentaria_temp.columns = ['Receitas Orçamentárias','Previsao Inicial','Previsao Atualizada (a)','Receitas Realizadas no Bimestre (b)','% (b/a)','Receitas Realizadas até o Bimestre (c)','% (c/a)','SALDO (a-c)']
        receita_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria_temp['Bimestre'] = ['{}'.format(i)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria_temp['Ano'] = ['{}'.format(j)]*len(receita_orcamentaria_temp['Receitas Orçamentárias'])
        receita_orcamentaria = pd.concat([receita_orcamentaria_temp,receita_orcamentaria])
        
receita_orcamentaria = arruma_data(receita_orcamentaria)
receita_orcamentaria.to_csv(path_download3, index = True)



Começando a processar o bimestre 01/2021


HTTPError: HTTP Error 404: Not Found

In [6]:
lista=['RECEITAS CORRENTES',
        'Impostos',
        'RECEITA PATRIMONIAL',
        'Contribuições Sociais',
        'RECEITA DE SERVIÇOS',
        'TRANSFERÊNCIAS CORRENTES',
        'OUTRAS RECEITAS CORRENTES']

df=pd.read_csv(path_download3)
df.set_index('Data',inplace=True)
df_final=pd.DataFrame()

for selected_income  in lista:
    filtered_df=df[df['Receitas Orçamentárias']==selected_income]
    filtered_df.fillna("0",inplace=True)
    filtered_df= arruma_num_rec(filtered_df)
    filtered_df = pd.concat([filtered_df,ipca],axis=1,join='outer')
    filtered_df = aplica_inflacao_rec(filtered_df) 
    filtered_df.sort_values(by=['tempo'],inplace=True)
    df_final=df_final.append(filtered_df)
df_final.to_csv(path_download4_1)

## Receitas por função

In [29]:
lista2 = ['RECEITAS CORRENTES','Impostos', 'Taxas',
       'Contribuição de Melhoria', 
       'Contribuições Sociais', 
       'Exploração do Patrimônio Imobiliário do Estado',
       'Valores Mobiliários', 
       'Serviços Administrativos e Comerciais Gerais', 
       'Transferências da União e de suas Entidades',
       'Transferências dos Estados e do Distrito Federal e de suas Entidades',
       'Transferências dos Municípios e de suas Entidades',
       'Transferências de Outras Instituições Públicas',
       'Multas Administrativas, Contratuais e Judiciais',
       'Indenizações, Restituições e Ressarcimentos',
       'Bens, Direitos e Valores Incorporados ao Patrimônio Público',
       'Demais Receitas Correntes', 'Operações de Crédito - Mercado Interno',
       'Alienação de Bens Móveis']
df=pd.read_csv(path_download3)
df.set_index('Data',inplace=True)
df=df[df['Ano']>2018]
df_final=pd.DataFrame()

for selected_income  in lista2:
    filtered_df=df[df['Receitas Orçamentárias']==selected_income]
    filtered_df.fillna("0",inplace=True)
    filtered_df= arruma_num_rec(filtered_df)
    if len(filtered_df)>12:
        filtered_df.reset_index(inplace=True)
        filtered_df.reset_index(inplace=True)
        filtered_df['eh_par']=filtered_df['index'].apply(eh_par)
        filtered_df = filtered_df[filtered_df['eh_par']==1]
        filtered_df.set_index('Data',inplace=True)
        filtered_df.drop(['index','eh_par'],axis=1,inplace=True)
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências dos Estados e do Distrito Federal e de suas Entidades':
        filtered_df['Receitas Orçamentárias'] = ['Transf. Estados/DF']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências da União e de suas Entidades':
        filtered_df['Receitas Orçamentárias'] = ['Transf. União']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Serviços Administrativos e Comerciais Gerais':
        filtered_df['Receitas Orçamentárias'] = ['Serv. Admin. ']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Multas Administrativas, Contratuais e Judiciais':
        filtered_df['Receitas Orçamentárias'] = ['Multas']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Transferências de Outras Instituições Públicas':
        filtered_df['Receitas Orçamentárias'] = ['Transf. Outras Inst.']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Operações de Crédito - Mercado Interno':
        filtered_df['Receitas Orçamentárias'] = ['Op. Crédito - Merc. Int.']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Exploração do Patrimônio Imobiliário do Estado':
        filtered_df['Receitas Orçamentárias'] = ['Rec. Imobiliária']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Contribuições Sociais':
        filtered_df['Receitas Orçamentárias'] = ['Contr. Sociais']*len(filtered_df['Receitas Orçamentárias'])
    if filtered_df['Receitas Orçamentárias'].iloc[0]== 'Demais Receitas Correntes':
        filtered_df['Receitas Orçamentárias'] = ['Demais Rec.']*len(filtered_df['Receitas Orçamentárias'])
    
    filtered_df = pd.concat([filtered_df,ipca.iloc[18:]],axis=1,join='outer')
    filtered_df = aplica_inflacao_rec(filtered_df) 
    filtered_df.sort_values(by=['tempo'],inplace=True)
    df_final=df_final.append(filtered_df)
df_final.to_csv(path_download4_2)

/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-29-ed504a59e776>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

# Descriminação dos Impostos no ano 2020

In [12]:
listaimposto=['IPTU','ISS','ITBI','IRRF']
listatransf=['Cota-Parte do FPM','Cota-Parte do ICMS','Cota-Parte do IPVA','Cota-Parte do ITR','Transferências do FUNDEB']

file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/2021-02RREO6Bim-declaracao.pdf'
todo = camelot.read_pdf(file,pages='16')
receita = todo[1].df
receita.columns = ['Tipo','janeiro 2020','fevereiro 2020','março 2020','abril 2020','maio 2020',
                   'junho 2020','julho 2020','agosto 2020','setembro 2020','outubro 2020',
                   'novembro 2020','dezembro 2020','Total','Prev']
receita=receita.iloc[4:,:]
receita=receita.T
receita.columns = receita.iloc[0]
receita = receita.iloc[1:-2]
receita = receita.iloc[:,:-10]
receita = receita.join(ipca_full.iloc[-13:-1])
receita.drop(['Transferências da LC nº 87/1996','Receita Agropecuária','Receita Industrial'],axis=1,inplace=True)
receita=arruma_num_rec_imp(receita)
receita=aplica_inflacao_rec_imp(receita)
receita.index=['Jan','Fev','Mar','Abr', 'Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
receita.to_csv(path_download4_3)

imposto= receita[listaimposto]
trans=receita[listatransf]
imposto.to_csv(path_download4_3_1)
trans.to_csv(path_download4_3_2)

In [14]:
imposto

,IPTU,ISS,ITBI,IRRF
Jan,2.42,17.97,3.37,5.41
Fev,6.52,15.37,8.16,4.64
Mar,55.66,15.44,2.48,4.80
Abr,9.16,15.54,1.83,4.86
Mai,9.15,14.13,2.66,4.98
Jun,9.76,12.32,2.69,4.91
Jul,9.76,17.21,3.89,4.83
Ago,9.62,18.01,3.79,4.94
Set,9.55,17.15,5.22,4.82
Out,9.17,16.08,6.59,4.75


In [17]:
trans=pd.read_csv(path_download4_3_2,index_col='Unnamed: 0')
trans


,Cota-Parte do FPM,Cota-Parte do ICMS,Cota-Parte do IPVA,Cota-Parte do ITR,Transferências do FUNDEB
Jan,6.28,36.95,41.80,0.11,19.88
Fev,9.05,37.80,17.93,0.03,16.78
Mar,5.29,43.54,14.13,0.03,17.43
Abr,5.20,29.50,1.77,0.02,10.78
Mai,5.46,29.11,2.12,0.03,10.82
Jun,4.43,35.42,3.64,0.01,13.04
Jul,7.84,31.29,3.08,0.01,11.60
Ago,4.82,37.43,3.44,0.04,13.62
Set,3.86,43.46,4.45,0.38,15.74
Out,5.15,41.71,3.46,1.49,15.22


## Despesa por bimestre

In [46]:
bimestre = ['01','02','03','04','05','06']
ano = ['2020','2019','2018','2017','2016']
despesa_orcamentaria=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j) ==2020:
            if int(i)==6:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/2021-02RREO6Bim-declaracao.pdf'
            else:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        else:
            file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        if j=='2016':
            todo=camelot.read_pdf(file,pages='2,3')
            despesa_orcamentaria_temp= todo[2].df.iloc[4:]
            despesa_orcamentaria_temp=pd.concat([despesa_orcamentaria_temp,todo[4].df.iloc[3:]])
        else:
            todo = camelot.read_pdf(file,pages='3')
            despesa_orcamentaria_temp=todo[1].df.iloc[2:]
            
        despesa_orcamentaria_temp.drop(10,axis=1,inplace=True)
        despesa_orcamentaria_temp.columns = ['Despesas Orçamentárias','DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE','DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)','SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE','DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
        despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria_temp['Bimestre'] = ['{}'.format(i)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria_temp['Ano'] = ['{}'.format(j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
        despesa_orcamentaria = pd.concat([despesa_orcamentaria_temp,despesa_orcamentaria])
        
despesa_orcamentaria = arruma_data(despesa_orcamentaria)
despesa_orcamentaria = despesa_orcamentaria[(despesa_orcamentaria['Despesas Orçamentárias']!='') & (despesa_orcamentaria['Despesas Orçamentárias']!='Despesas Orçamentárias')]
despesa_orcamentaria.to_csv(path_download6, index = True)


Começando a processar o bimestre 01/2020


/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-46-03f2a055c6c8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-46-03f2a055c6c8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Começando a processar o bimestre 02/2020


/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-46-03f2a055c6c8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-46-03f2a055c6c8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Começando a processar o bimestre 03/2020


...

Começando a processar o bimestre 05/2017


/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-46-03f2a055c6c8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-46-03f2a055c6c8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Começando a processar o bimestre 06/2017


/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-46-03f2a055c6c8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  despesa_orcamentaria_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_orcamentaria_temp.loc[:,'Despesas Orçamentárias'])
<ipython-input-46-03f2a055c6c8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Começando a processar o bimestre 01/2016
Começando a processar o bimestre 02/2016
Começando a processar o bimestre 03/2016
Começando a processar o bimestre 04/2016
Começando a processar o bimestre 05/2016
Começando a processar o bimestre 06/2016


In [79]:
lista=['DESPESAS CORRENTES',
        'PESSOAL E ENCARGOS SOCIAIS',
        'JUROS E ENCARGOS DA DÍVIDA',
        'OUTRAS DESPESAS CORRENTES',
        'INVESTIMENTOS',
        'AMORTIZAÇÃO DA DÍVIDA']
df=pd.read_csv(path_download6)
df.set_index('Data',inplace=True)

df_final=pd.DataFrame()
filtered_df =pd.DataFrame()
for selected_income  in lista:
    filtered_df=df[df['Despesas Orçamentárias']==selected_income]
    filtered_df=arruma_num_des(filtered_df)
    filtered_df = pd.concat([filtered_df,ipca],axis=1,join='outer')
    filtered_df = aplica_inflacao_des(filtered_df)
    filtered_df.sort_values(by=['tempo'],inplace=True)
    df_final=df_final.append(filtered_df)
df_final.to_csv(path_download7)

/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Despesa por função

In [38]:
datas=[3,5,7]
bimestre = ['01','02','03','04','05','06']
ano = ['2020','2019','2018']
despesa=pd.DataFrame()
for j in ano:
    for i in bimestre:
        print("Começando a processar o bimestre {}/{}".format(i,j))
        if int(j) ==2020:
            if int(i)==6:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/2021-02RREO6Bim-declaracao.pdf'
            else:
                file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        else:
            file = 'http://www.financas.piracicaba.sp.gov.br/upload/kceditor/files/relat_res_{}_bim_{}.pdf'.format(i,j)
        
        
        
        todo = camelot.read_pdf(file,pages='7,8,9,10')
        despesa_temp=todo[1].df.iloc[3:]
        for d in datas:
            despesa_temp=despesa_temp.append(todo[d].df.iloc[3:])
        despesa_temp.columns = ['Despesas Orçamentárias','DOTAÇÃO INICIAL (d)','DOTAÇÃO ATUALIZADA (e)','DESPESAS EMPENHADAS NO BIMESTRE','DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)',('% (b/total b)'),'SALDO (g) =(e-f)','DESPESAS LIQUIDADAS NO BIMESTRE','DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)','% (d/total d)','SALDO (i) = (e-h)','DESPESAS PAGASATÉ O BIMESTRE(j)']
        despesa_temp['Data'] = ['{}/{}'.format(i,j)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa_temp['Bimestre'] = ['{}'.format(i)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa_temp['Ano'] = ['{}'.format(j)]*len(despesa_temp.loc[:,'Despesas Orçamentárias'])
        despesa = pd.concat([despesa_temp,despesa])
        
despesa = arruma_data(despesa)
despesa = despesa[(despesa['Despesas Orçamentárias']!='') & (despesa['Despesas Orçamentárias']!='Despesas Orçamentárias')]
despesa.to_csv(path_download9, index = True)




Começando a processar o bimestre 01/2020
Começando a processar o bimestre 02/2020
Começando a processar o bimestre 03/2020
Começando a processar o bimestre 04/2020
Começando a processar o bimestre 05/2020
Começando a processar o bimestre 06/2020
Começando a processar o bimestre 01/2019
Começando a processar o bimestre 02/2019
Começando a processar o bimestre 03/2019
Começando a processar o bimestre 04/2019
Começando a processar o bimestre 05/2019
Começando a processar o bimestre 06/2019
Começando a processar o bimestre 01/2018
Começando a processar o bimestre 02/2018
Começando a processar o bimestre 03/2018
Começando a processar o bimestre 04/2018
Começando a processar o bimestre 05/2018
Começando a processar o bimestre 06/2018


In [39]:
listaTopo=['Legislativa',#Topo
    'Essencial à Justiça', #Topo
    'Administração',#Topo
    'Segurança Pública',#Topo
    'Assistência Social',#Topo
    'Previdência Social',#Topo
    'Saúde',#Topo
    'Educação',#Topo
    'Trabalho',#Topo
    'Cultura',#Topo
    'Direitos da Cidadania',#Topo
    'Urbanismo',#Topo
    'Habitação',#Topo
    'Saneamento',#Topo
    'Gestão Ambiental',#Topo
    'Ciência e Tecnologia',#Topo
    'Agricultura',#Topo
    'Turismo',#Topo
    'Transporte',#Topo
    'Desporto e Lazer',#Topo
    'Encargos Especiais'#Topo]
        ]
listaAdm=['Planejamento e Orçamento', 'Administração Financeira',
        'Controle Interno', 'Normatização e Fiscalização',
        'Tecnologia da Informação', 'Ordenamento Territorial',
        'Formação de Recursos Humanos', 'Administração de Receitas',
        'Administração de Concessões', 'Comunicação Social','FU04 - Administração Geral']

listSeg = ['Policiamento','Defesa Civil', 'Informação e Inteligência']

listaAss = ['Assistência ao Idoso','Assistência ao Portador de Deficiência',
        'Assistência à Criança e ao Adolescente','Assistência Comunitária']
listaSaude=['Atenção Básica', 'Assistência Hospitalar e Ambulatorial',
        'Suporte Profilático e Terapêutico', 'Vigilância Sanitária',
        'Vigilância Epidemiológica', 'Alimentação e Nutrição','FU10 - Administração Geral']
listaEdu=['Ensino Fundamental', 'Ensino Médio', 'Ensino Profissional',
        'Ensino Superior', 'Educação Infantil',
        'Educação de Jovens e Adultos', 'Educação Especial',
        'Educação Básica','FU12 - Demais Subfunções','FU12 - Administração Geral']
listaTrab=['Proteção e Benefícios ao Trabalhador', 'Relações de Trabalho',
        'Empregabilidade', 'Fomento ao Trabalho']       
listaCult = ['Patrimônio Histórico Artístico e Arqueológico',
        'Difusão Cultural'] 
listaUrb=['Infra-Estrutura Urbana', 'Serviços Urbanos',
        'Transportes Coletivos Urbanos']
listaSan=['Saneamento Básico Rural',
       'Saneamento Básico Urbano','FU17 - Administração Geral','FU17 - Demais Subfunções',]
listaGesAmb=[ 'Preservação e Conservação Ambiental', 'Controle Ambiental',
       'Recuperação de Áreas Degradadas', 'Recursos Hídricos',
        'Meteorologia','FU18 - Administração Geral' ]
listaAgr=['Abastecimento', 'Extensão Rural', 'Irrigação',
        'Promoção da Produção Agropecuária', 'Defesa Agropecuária','FU20 - Administração Geral']
listaDesp=['Desporto de Rendimento', 'Desporto Comunitário', 'Lazer','FU27 - Administração Geral']

listaListas = [listaTopo,listaAdm,listSeg,listaAss,listaSaude,listaEdu,
             listaTrab,listaCult,listaUrb,listaSan,listaGesAmb,listaAgr,listaDesp]

i=0
df=pd.read_csv(path_download9)
df.set_index('Data',inplace=True)

for lista  in listaListas:
    df_final=pd.DataFrame()
    filtered_df =pd.DataFrame()
    for item in lista:
        filtered_df=df[df['Despesas Orçamentárias']==item]
        filtered_df=arruma_num_des_fun(filtered_df)
        filtered_df.drop('DESPESAS PAGASATÉ O BIMESTRE(j)',axis=1,inplace=True)
        filtered_df.fillna(0,inplace=True)
        filtered_df = pd.concat([filtered_df,ipca.iloc[12:]],axis=1,join='outer')
        filtered_df = aplica_inflacao_des_fun(filtered_df)
        filtered_df.sort_values(by=['tempo'],inplace=True)
        df_final=df_final.append(filtered_df)
    i = i + 1
    df_final.to_csv(Path(path_download10,'{}.csv'.format(i)), index = True)
    

/home/yan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/yan/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

## Votos na Câmara

In [10]:
base_df = pd.DataFrame()
num={'457925':datetime.strptime('04/02/21', '%d/%m/%y').date(),'459087':datetime.strptime('08/02/21', '%d/%m/%y').date(),
     '459636':datetime.strptime('11/02/21', '%d/%m/%y').date(),'459751':datetime.strptime('15/02/21', '%d/%m/%y').date(),
     '461101':datetime.strptime('22/02/21', '%d/%m/%y').date(),'461060':datetime.strptime('25/02/21', '%d/%m/%y').date(),
     '461170':datetime.strptime('01/03/21', '%d/%m/%y').date()}
for j,k in num.items():
    print(j)
    file = 'https://siave.camarapiracicaba.sp.gov.br/arquivo?Id={}'.format(j)
    todo = camelot.read_pdf(file,pages='all')
    for i in range(len(todo)):
        print('{}/{}'.format(j,i))
        if todo[i].df.iloc[0,0].split('\n')[6]!='0':
            vereador = todo[i].df.iloc[0,0].split('\n')[15]
            if len(todo[i].df.iloc[0,0].split('\n'))<=29:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]
            elif len(todo[i].df.iloc[0,0].split('\n'))==30:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]
            elif len(todo[i].df.iloc[0,0].split('\n'))==31:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]
            elif len(todo[i].df.iloc[0,0].split('\n'))==32:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]
            elif len(todo[i].df.iloc[0,0].split('\n'))==33:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]+ todo[i].df.iloc[0,0].split('\n')[22]
            presentes = int(todo[i].df.iloc[0,1].split('\n')[0])
            ausentes = int(todo[i].df.iloc[0,1].split('\n')[2])
            if len(todo[i].df.iloc[1,1].split('\n')[0])<3:
                sim = int(todo[i].df.iloc[1,1].split('\n')[0])
                nao = int(todo[i].df.iloc[1,1].split('\n')[2])
                abst= int(todo[i].df.iloc[1,1].split('\n')[4])
            else:
                sim = int(todo[i].df.iloc[1,1].split('\n')[1])
                nao = int(todo[i].df.iloc[1,1].split('\n')[0].split()[0])
                abst= int(todo[i].df.iloc[1,1].split('\n')[0].split()[1])
            status=todo[i].df.iloc[2,1]
            tempo_vot = todo[i].df.iloc[0,0].split('\n')[-3]
            dicionario = {'Data':k,'Propositor':vereador,'Ementa':ementa,'Presentes':presentes,
                        'Votos a Favor':sim,'Votos Contrários':nao, 'Status':status,'Tempo de Votação':tempo_vot}
            base_df=base_df.append(dicionario,ignore_index=True)
        else:
            continue
base_df.to_csv(Path(path_download10,'sessoes.csv'),index=False)

457925
457925/0
457925/1
457925/2
457925/3
457925/4
457925/5
457925/6
457925/7
457925/8
457925/9
457925/10
457925/11
457925/12
457925/13
457925/14
457925/15
457925/16
457925/17
457925/18
457925/19
457925/20
457925/21
457925/22
457925/23
457925/24
457925/25
457925/26
457925/27
457925/28
457925/29
457925/30
457925/31
457925/32
457925/33
457925/34
457925/35
457925/36
457925/37
457925/38
457925/39
457925/40
457925/41
457925/42
457925/43
457925/44
457925/45
457925/46
457925/47
457925/48
457925/49
457925/50
457925/51
457925/52
457925/53
457925/54
457925/55
457925/56
457925/57
457925/58
457925/59
457925/60
457925/61
457925/62
457925/63
457925/64
457925/65
457925/66
457925/67
457925/68
457925/69
457925/70
457925/71
457925/72
457925/73
457925/74
457925/75
457925/76
457925/77
457925/78
457925/79
457925/80
457925/81
457925/82
457925/83
457925/84
457925/85
457925/86
457925/87
457925/88
457925/89
457925/90
457925/91
457925/92
457925/93
457925/94
457925/95
457925/96
457925/97
457925/98
457925/99
457

## Atualizar a base dia a dia

In [8]:
base_df = pd.read_csv(Path(path_download10,'sessoes.csv'))
num={'461170':'01/03/21'}
for j,k in num.items():
    print(j)
    file = 'https://siave.camarapiracicaba.sp.gov.br/arquivo?Id={}'.format(j)
    todo = camelot.read_pdf(file,pages='all')
    for i in range(len(todo)):
        print('{}/{}'.format(j,i))
        if todo[i].df.iloc[0,0].split('\n')[6]!='0':
            vereador = todo[i].df.iloc[0,0].split('\n')[15]
            if len(todo[i].df.iloc[0,0].split('\n'))<=29:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]
            elif len(todo[i].df.iloc[0,0].split('\n'))==30:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]
            elif len(todo[i].df.iloc[0,0].split('\n'))==31:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]
            elif len(todo[i].df.iloc[0,0].split('\n'))==32:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]
            elif len(todo[i].df.iloc[0,0].split('\n'))==33:
                ementa = todo[i].df.iloc[0,0].split('\n')[17]+todo[i].df.iloc[0,0].split('\n')[18]+ todo[i].df.iloc[0,0].split('\n')[19]+ todo[i].df.iloc[0,0].split('\n')[20]+ todo[i].df.iloc[0,0].split('\n')[21]+ todo[i].df.iloc[0,0].split('\n')[22]
            presentes = int(todo[i].df.iloc[0,1].split('\n')[0])
            ausentes = int(todo[i].df.iloc[0,1].split('\n')[2])
            if len(todo[i].df.iloc[1,1].split('\n')[0])<3:
                sim = int(todo[i].df.iloc[1,1].split('\n')[0])
                nao = int(todo[i].df.iloc[1,1].split('\n')[2])
                abst= int(todo[i].df.iloc[1,1].split('\n')[4])
            else:
                sim = int(todo[i].df.iloc[1,1].split('\n')[1])
                nao = int(todo[i].df.iloc[1,1].split('\n')[0].split()[0])
                abst= int(todo[i].df.iloc[1,1].split('\n')[0].split()[1])
            status=todo[i].df.iloc[2,1]
            tempo_vot = todo[i].df.iloc[0,0].split('\n')[-3]
            dicionario = {'Data':k,'Propositor':vereador,'Ementa':ementa,'Presentes':presentes,
                        'Votos a Favor':sim,'Votos Contrários':nao, 'Status':status,'Tempo de Votação':tempo_vot}
            base_df=base_df.append(dicionario,ignore_index=True)
        else:
            continue
base_df.to_csv(Path(path_download10,'sessoes.csv'),index=False)

461101
461101/0
461101/1
461101/2
461101/3
461101/4
461101/5
461101/6
461101/7
461101/8
461101/9
461101/10
461101/11
461101/12
461101/13
461101/14
461101/15
461060
461060/0
461060/1
461060/2
461060/3
461060/4
461060/5
461060/6
461060/7
461060/8
461060/9
461060/10
461060/11
461060/12
461060/13
461060/14
461060/15
461060/16
461060/17
461060/18
461060/19
461060/20
461060/21
461060/22
461060/23
461060/24
461060/25
461060/26
461060/27
461060/28
461060/29
461060/30
461170
461170/0
461170/1
461170/2
461170/3
461170/4
461170/5
461170/6
461170/7
461170/8
461170/9
461170/10
461170/11
461170/12
461170/13
461170/14
461170/15


## Esse codigo ainda não foi utilizado:

In [ ]:
df_final=pd.DataFrame()
dicionario={'ADRIANA C. SGRIGNEIRO NUNES':27817,
           'ALDISA VIEIRA MARQUES': 27818,
           'ANDRÉ GUSTAVO BANDEIRA': 648,
           'ARY DE CAMARGO PEDROSO JUNIOR': 447,
           'CARLOS GOMES DA SILVA': 448,
           'DIRCEU ALVES DA SILVA': 659,
           'GILMAR ROTTA':665,
           'ISAC ALVES DE SOUZA': 27819,
           'JONSON S. DE OLIVEIRA':27820,
           'JOSÉ A. LONGATTO': 420,
           'JOSÉ FRANCISCO RONCATO': 27873,
           'JOSÉ MARCOS ABDALA':27821,
           'LAÉRCIO TREVISAN JR': 652,
           'LAIR BRAGA': 27822,
           'MATHEUS ANTONIO ERLER': 661,
           'NANCY A. F. THAME':27823,
           'OSVALDO A. SCHIAVOLIN':27824,
           'PAULO E. SEADE SERRA':671,
           'PAULO HENRIQUE PARANHOS RIBEIRO':651,
           'PAULO ROBERTO CAMPOS':668,
           'PEDRO MOTOITIRO KAWAI':666,
           'RERLISON T. DE REZENDE':27825,
           'RONALDO MOSCHINI SILVA':657,
           'WAGNER A. DE OLIVEIRA':27826,
           }
for ver in dicionario.items():
    print(ver[0], ver[1])
    url = 'https://siave.camarapiracicaba.sp.gov.br/Vereadores/Contagem/27817{}'.format(ver[1])
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df  = df_list[0].T.reset_index()
    df.drop('level_0',axis=1,inplace=True)
    df.columns=df.iloc[0]
    df=df.iloc[1:]
    df = df.melt(id_vars=['Tipo'], value_vars=df.columns[1:])
    df['Vereador']=ver[0]
    df_final=df_final.append(df,ignore_index=True)
df_final.columns=['Ano','Tipo','Valor','Vereador(a)']
df_final['Valor']=df_final['Valor'].apply(acha_string)

In [ ]:
df_final[(df_final['Tipo']=='Indicações') &(df_final['Ano']=='Total')]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0]}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0]}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 1}))
fig.add_trace(go.Indicator(
    value = 269,
    delta = {'reference': 300},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 1, 'column': 1}))


fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Força"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})

## Mapas Vereadores 2020

In [ ]:
mapa = pd.read_csv(path_download13,sep=',',encoding = "ISO-8859-1")
locais=pd.read_csv(path_download14)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]

dicionario={'André Bandeira':'ANDRE GUSTAVO BANDEIRA','Alessandra Protetora Animais':'Alessandra Bellucci'.upper(),
            'Paulo Camolesi':'Paulo Sergio Camolesi'.upper(),'Acácio Godoy':'ACÃCIO GERALDO SOUZA DE GODOY',
            'Ana Lúcia Pavão':'ANA LUCIA BATISTA PAVÃO','Paraná':'Aldisa Vieira Marques'.upper(),
            'Anilton Rissato':'Anilton Fernandes Rissato'.upper(), 'Ary Pedroso Jr':'Ary de Camargo Pedroso Junior'.upper(),
            'Cássio Luiz Fala Pira':'Cassio Luiz Barbosa'.upper(),'Fabricio Polezi':'FABRICIO JOSÃ RAETZ DE OLIVEIRA POLEZI',
            'Gilmar Rotta':'Gilmar Rotta'.upper(),'Gustavo Pompeo':'Gustavo Pompeo'.upper(),
            'Zezinho Pereira':'JOSÃ ANTONIO PEREIRA','Josef Borges':'Jose Everaldo Borges'.upper(),
            'Laércio Trevisan Jr':'Laercio Trevisan Junior'.upper(),'Paulo Henrique Ribeiro':'Paulo Henrique Paranhos Ribeiro'.upper(),
            'Paulo Roberto de Campos':'Paulo Roberto de Campos'.upper(),'Pedro Kawai':'Pedro Motoitiro Kawai'.upper(),
            'Rai de Almeida':'Raimunda Ferreira de Almeida'.upper(), 'Relinho':'Rerlison Teixeira de Rezende'.upper(),
            'Silvia Mandato Coletivo':'Silvia Maria Morales'.upper(),'Thiago Augusto Ribeiro':'Thiago Augusto Ribeiro'.upper(),
            'Wagnão':'Wagner Alexandre de Oliveira'.upper(),'Cimarinha':'CIMARA PEREIRA PRADA'
           }



ver_eleitos=pd.DataFrame()
ver_eleitos = vereador[vereador['NM_VOTAVEL'].isin(dicionario.values())]



ver_eleitos.loc[:,'Posicao'] = ver_eleitos.loc[:,'NR_SECAO'].apply(retorna_posicao)
ver_eleitos.loc[:,'Longitude'] = ver_eleitos.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
ver_eleitos.loc[:,'Latitude'] = ver_eleitos.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in ver_eleitos['NR_SECAO'].unique():
    df_temp=ver_eleitos[ver_eleitos['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
ver_eleitos['Votos_Totais_Secao']=ver_eleitos['NR_SECAO'].apply(votos_totais_secao)
ver_eleitos.loc[:,'% votos da sessao']=(ver_eleitos.loc[:,'QT_VOTOS']/ver_eleitos.loc[:,'Votos_Totais_Secao'])*100

for item in dicionario.items():
    ver_eleito = ver_eleitos[ver_eleitos['NM_VOTAVEL']==item[1]]
    parent = pathlib.Path().absolute()
    path_download13 = Path(parent,"bases_brutas/{}/eleicao/vereadores/mapa/votacao.csv".format(cidade))
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Camara/Vereadores/Vereadores/{}.csv'.format(item[0]))
    ver_eleito.to_csv(path_download_especial)

for nome in dicionario.keys():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Camara/Vereadores/Vereadores/{}.csv'.format(nome))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False)
    nova = nova[nova['% votos da sessao']>7]
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=11)
    mapa.add_child(plugins.HeatMap(coordenadas))
    mapa.save('mapa_ver_{}.html'.format(nome))

## Mapas Prefeitos 1T/2020

In [8]:
mapa = pd.read_csv(path_download13,sep=',',encoding = "ISO-8859-1")
votacao = pd.read_csv(path_download14,sep=',',encoding = "ISO-8859-1")
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]


dicionario={'Nancy Thame':'NANCY APARECIDA FERRUZI THAME',
            'Francys Almeida':'FRANCYS ALMEIDA DA SILVA'.upper(),
            'Luciano Almeida':'Luciano Santos Tavares de Almeida'.upper(),
            'Professor Adelino':'Adelino Francisco de Oliveira'.upper(),
            'Mário Neto':'MÃ\x81RIO MEDEIROS NETO',
            'Zé Pedro':'JOSÃ\x89 PEDRO LEITE DA SILVA',
            'Érica Gorga':'Ã\x89RICA CRISTINA ROCHA GORGA'.upper(),
            'Barjas Negri':'BARJAS NEGRI'.upper(),
            'Carlito':'JOSE CARLOS LOPES DE SOUZA'.upper(),
            'Carolina Angelelli':'CAROLINA ANGELELLI'.upper(),
            'Coronel Adriana':'ADRIANA CRISTINA SGRIGNEIRO NUNES'.upper(),
            'Edvaldo Brito':'JOSE EDVALDO BRITO'.upper(),    
            }



locais=pd.read_csv(path_download14)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)

pref_prim_turno.loc[:,'Posicao'] = pref_prim_turno.loc[:,'NR_SECAO'].apply(retorna_posicao)
pref_prim_turno.loc[:,'Longitude'] = pref_prim_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
pref_prim_turno.loc[:,'Latitude'] = pref_prim_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in pref_prim_turno['NR_SECAO'].unique():
    df_temp=pref_prim_turno[pref_prim_turno['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
pref_prim_turno['Votos_Totais_Secao']=pref_prim_turno['NR_SECAO'].apply(votos_totais_secao)
pref_prim_turno.loc[:,'% votos da sessao']=(pref_prim_turno.loc[:,'QT_VOTOS']/pref_prim_turno.loc[:,'Votos_Totais_Secao'])*100

for nome in dicionario.items():
    pref_prim_turno_temp = pref_prim_turno[pref_prim_turno['NM_VOTAVEL']==nome[1]]
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome[0]))
    pref_prim_turno_temp.to_csv(path_download_especial)
    
for nome in dicionario.items():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome[0]))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False).head(60)
    
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=11)
    mapa.add_child(plugins.HeatMap(coordenadas))        
    mapa.save('mapa_pref_{}.html'.format(nome[0]))

/home/yan/anaconda3/envs/Yan/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/yan/anaconda3/envs/Yan/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-8-03199f85dc39>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [ ]:
nome='BARJAS NEGRI'
path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/1T/{}.csv'.format(nome))
df=pd.read_csv(path_download_especial_1)
nova=df.sort_values('% votos da sessao',ascending=False)
nova

## Mapas Prefeitos 2T/2020

In [ ]:
mapa = pd.read_csv(path_download13,sep=',',encoding = "ISO-8859-1")
votacao = pd.read_csv(path_download14,sep=',',encoding = "ISO-8859-1")
prefeito = mapa[mapa['DS_CARGO']=='PREFEITO']
vereador = mapa[mapa['DS_CARGO']=='VEREADOR']
pref_prim_turno = prefeito[prefeito['NR_TURNO']==1]
pref_seg_turno = prefeito[prefeito['NR_TURNO']==2]


locais=pd.read_csv(path_download14)
locais['Seções2'] = locais['Seções'].apply(numeros_sessoes)

pref_seg_turno.loc[:,'Posicao'] = pref_seg_turno.loc[:,'NR_SECAO'].apply(retorna_posicao)
pref_seg_turno.loc[:,'Longitude'] = pref_seg_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[1])
pref_seg_turno.loc[:,'Latitude'] = pref_seg_turno.loc[:,'Posicao'].apply(lambda x: x.split(',')[0])


df=pd.DataFrame()
for i in pref_seg_turno['NR_SECAO'].unique():
    df_temp=pref_seg_turno[pref_seg_turno['NR_SECAO']==i]
    dici={'Seção':i,'Votos':df_temp.QT_VOTOS.sum()}
    df=df.append(dici,ignore_index=True)
pref_seg_turno['Votos_Totais_Secao']=pref_seg_turno['NR_SECAO'].apply(votos_totais_secao)
pref_seg_turno.loc[:,'% votos da sessao']=(pref_seg_turno.loc[:,'QT_VOTOS']/pref_seg_turno.loc[:,'Votos_Totais_Secao'])*100

for nome in pref_seg_turno.NM_VOTAVEL.unique():
    pref_seg_turno_temp = pref_seg_turno[pref_seg_turno['NM_VOTAVEL']==nome]
    path_download_especial = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/2T/{}.csv'.format(nome))
    pref_seg_turno_temp.to_csv(path_download_especial)
    
for nome in pref_seg_turno.NM_VOTAVEL.unique():
    path_download_especial_1 = Path(path_download,'Site_Pira/Bases_Brutas/Prefeitura/Eleicao/2T/{}.csv'.format(nome))
    df=pd.read_csv(path_download_especial_1)
    nova=df.sort_values('% votos da sessao',ascending=False).head(200)
    coordenadas=[]
    for lat,lng in zip(nova.Latitude.values,nova.Longitude.values):
      coordenadas.append([lat,lng])
    mapa = folium.Map(location=[-22.732391682348542, -47.647584031127465],zoom_start=14)
    mapa.add_child(plugins.HeatMap(coordenadas))        
    mapa.save('mapa_pref_{}.html'.format(nome))

# COVID Piracicaba

## Base primária

In [14]:
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)
dias = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
meses =['abril','maio','junho','julho','agosto','setembro','outubro']
df = pd.DataFrame()
for m in meses:
    for d in dias:
        if (m == 'outubro') & (d=='23'):
            link = 'http://www.piracicaba.sp.gov.br/coronavirus+atualizacao+de+casos+em+23+10.aspx'
        elif (m == 'abril') & (d=='22'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+22+de+abril+de+2020+16h+65975.aspx'
        elif (m == 'abril') & (d=='23'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+23+de+abril+de+2020+16h15.aspx'
        elif (m == 'abril') & (d=='24'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+24+de+abril+de+2020+15h40.aspx'
        elif (m == 'maio') & (d=='10'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+10+de+maio+de+2020+16h30.aspx'
        elif (m == 'maio') & (d=='17'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+17+de+maio+de+2020+17h.aspx'
        elif (m == 'maio') & (d=='21'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+21+de+maio+de+2020.aspx'
        elif (m == 'maio') & (d=='30'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+30+de+maio+de+2020.aspx' 
        elif (m == 'junho') & (d=='12'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+12+de+junho+de+2020+16h+66394.aspx'
        elif (m == 'setembro') & (d=='21'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+21+de+setembro+de+2020+as+16h.aspx'
        elif (m == 'outubro') & (d=='16'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+16+de+outubro+de+2010+16h.aspx'
        elif (m == 'agosto') & (d=='01'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+1+de+agosto+de+2020+16h.aspx'
        elif (m=='maio') & (d=='01'):
            link = 'http://www.piracicaba.sp.gov.br/comunicado+01+maio+de+2020+16h.aspx'
        else:
            link = 'http://www.piracicaba.sp.gov.br/comunicado+{}+de+{}+de+2020+16h.aspx'.format(d,m)
        print(link)
        driver.get(link)
        div = driver.find_element_by_id('imagenet-conteudo')
        novos_obitos = 'NA'
        casos_confirmados = 'NA'
        casos_suspeitos = 'NA'
        casos_descartados = 'NA'
        casos_recuperados = 'NA'
        em_tratamento = 'NA'
        obitos = 'NA'
        homens = 'NA'
        mulheres = 'NA'
        for i in range(len(div.text.split('\n'))):
            if (len(div.text.split('\n')[i].split())<5)&(len(div.text.split('\n')[i].split())>1):
                if div.text.split('\n')[i].split()[-1] =='confirmados':
                    casos_confirmados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='suspeitos':
                    casos_suspeitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='descartados':
                    casos_descartados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='recuperados':
                    casos_recuperados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='tratamento':
                    em_tratamento = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbitos':
                    obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbito':
                    obitos = div.text.split('\n')[i].split()[0]
            elif len(div.text.split('\n')[i].split())>1:
                if div.text.split('\n')[i].split()[1] =='homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher':
                    mulheres = div.text.split('\n')[i].split()[0]
        dicionario ={'Total Casos Confirmados': casos_confirmados, 'Total Casos Suspeitos':casos_suspeitos,
                    'Total Casos Descartados':casos_descartados,'Total Casos Recuperados':casos_recuperados,
                    'Total Casos em Tratamento':em_tratamento,'Total Óbitos':obitos,
                     'Homens':homens,'Mulheres':mulheres,'Data':'{}/{}/2020'.format(d,m)}
        df = df.append(dicionario, ignore_index=True)
driver.quit()
df.to_csv(path_download11)

http://www.piracicaba.sp.gov.br/comunicado+01+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+02+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+03+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+04+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+05+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+06+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+07+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+08+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+09+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+10+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+11+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+12+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+13+de+abril+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+14+de+abril+de+2020+1

http://www.piracicaba.sp.gov.br/comunicado+23+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+24+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+25+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+26+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+27+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+28+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+29+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+30+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+31+de+julho+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+1+de+agosto+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+02+de+agosto+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+03+de+agosto+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+04+de+agosto+de+2020+16h.aspx
http://www.piracicaba.sp.gov.br/comunicado+05+de+agosto+de+20

In [15]:
here = pathlib.Path().absolute()
path_win = Path(here,"chromedriver_ln")
driver = webdriver.Chrome(path_win)
dias = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
meses =['10','11','12','01','02']
df = pd.DataFrame()
for m in meses:
    for d in dias:
        if (m == '01'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2021.aspx'.format(d,m)
        elif (m == '02'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2021.aspx'.format(d,m)
        elif (m == '12')&(d=='24'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2020.aspx'.format(d,m)
        elif (m == '12')&(d=='25'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2020.aspx'.format(d,m)
        elif (m == '12')&(d=='26'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2020.aspx'.format(d,m)
        elif (m == '12')&(d=='27'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2020.aspx'.format(d,m)
        elif (m == '12')&(d=='31'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+2020.aspx'.format(d,m)
        elif (m == '01')&(d=='05'):
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+05+01+2021+67946.aspx'           
        else:
            link = 'http://www.piracicaba.sp.gov.br/plantao+coronavirus+{}+{}+20.aspx'.format(d,m)
        print(link)
        driver.get(link)
        div = driver.find_element_by_id('imagenet-conteudo')
        novos_obitos = 'NA'
        casos_confirmados = 'NA'
        casos_suspeitos = 'NA'
        casos_descartados = 'NA'
        casos_recuperados = 'NA'
        em_tratamento = 'NA'
        obitos = 'NA'
        homens = 'NA'
        mulheres = 'NA'
        for i in range(len(div.text.split('\n'))):
            if (len(div.text.split('\n')[i].split())<5)&(len(div.text.split('\n')[i].split())>1):
                if div.text.split('\n')[i].split()[-1] =='confirmados':
                    casos_confirmados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='suspeitos':
                    casos_suspeitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='descartados':
                    casos_descartados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='recuperados':
                    casos_recuperados = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='tratamento':
                    em_tratamento = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbitos':
                    obitos = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[-1] =='óbito':
                    obitos = div.text.split('\n')[i].split()[0]
            elif len(div.text.split('\n')[i].split())>1:
                if div.text.split('\n')[i].split()[1] =='homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homens':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='homem:':
                    homens = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher:':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='Mulheres':
                    mulheres = div.text.split('\n')[i].split()[0]
                elif div.text.split('\n')[i].split()[1] =='mulher':
                    mulheres = div.text.split('\n')[i].split()[0]
        dicionario ={'Total Casos Confirmados': casos_confirmados, 'Total Casos Suspeitos':casos_suspeitos,
                    'Total Casos Descartados':casos_descartados,'Total Casos Recuperados':casos_recuperados,
                    'Total Casos em Tratamento':em_tratamento,'Total Óbitos':obitos,
                     'Homens':homens,'Mulheres':mulheres,'Data':'{}/{}/2020'.format(d,m)}
        df = df.append(dicionario, ignore_index=True)
driver.quit()
df.to_csv(path_download12)

http://www.piracicaba.sp.gov.br/plantao+coronavirus+01+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+02+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+03+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+04+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+05+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+06+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+07+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+08+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+09+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+10+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+11+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+12+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+13+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+14+10+20.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+15+10+20.aspx
http://www

http://www.piracicaba.sp.gov.br/plantao+coronavirus+01+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+02+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+03+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+04+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+05+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+06+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+07+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+08+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+09+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+10+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+11+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+12+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+13+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavirus+14+02+2021.aspx
http://www.piracicaba.sp.gov.br/plantao+coronavi

In [67]:
covid1 = pd.read_csv(path_download11)
covid2 = pd.read_csv(path_download12)
covid1.dropna(how='all',thresh=8,inplace=True)
covid1.drop('Unnamed: 0',axis=1,inplace=True)
covid1.set_index('Data',inplace=True)
covid2.dropna(how='all',thresh=8,inplace=True)
covid2.drop('Unnamed: 0',axis=1,inplace=True)
covid2.set_index('Data',inplace=True)
covid=pd.concat([covid1,covid2],ignore_index=False)

In [107]:
covid = pd.read_csv(path_download15)
covid['Óbitos Diários'] = covid['Total Óbitos'] - covid['Total Óbitos'].shift(1)
covid['Casos Diários'] = covid['Total Casos Confirmados'] - covid['Total Casos Confirmados'].shift(1)
covid.to_csv(path_download15)

# Isolamento COVID-Pira

In [55]:
isolation=pd.read_csv(path_download20,';')

isolation.columns = ['Municipio', 'Codigo Municipio IBGE', 'Populacao estimada (2020)',
       'UF1', 'Data', 'Media_isolamento']
isolation=isolation[isolation['Municipio']=='PIRACICABA']
isolation['Data'] = isolation['Data'].apply(lambda x: x[-5:])
isolation['Media_isolamento']=isolation['Media_isolamento'].apply(lambda x: int(x[:-1]))
isolation['Data']=pd.to_datetime(isolation['Data'],format='%d/%m', errors='coerce')

In [62]:
isolation['Data'].iloc[0].day

1

In [56]:
isolation

,Municipio,Codigo Municipio IBGE,Populacao estimada (2020),UF1,Data,Media_isolamento
37156,PIRACICABA,3538709,407252,SP,1900-04-01,39
37157,PIRACICABA,3538709,407252,SP,1900-03-31,40
37158,PIRACICABA,3538709,407252,SP,1900-03-30,41
37159,PIRACICABA,3538709,407252,SP,1900-03-29,45
37160,PIRACICABA,3538709,407252,SP,1900-03-28,50
...,...,...,...,...,...,...
37543,PIRACICABA,3538709,407252,SP,1900-03-04,30
37544,PIRACICABA,3538709,407252,SP,1900-03-03,30
37545,PIRACICABA,3538709,407252,SP,1900-03-01,35
37546,PIRACICABA,3538709,407252,SP,NaT,32


In [51]:
data.month

4

In [48]:
data = datetime.strptime(isolation['Data'].iloc[0], '%d/%m')

# CAGED

Baixar o arquivo "tabelas" em http://pdet.mte.gov.br/novo-caged

# Despesas Câmara

In [3]:
ipca_full=pd.read_csv(path_download2_1,index_col='0')
desp = pd.read_excel(path_download16)
desp1 = pd.read_excel(path_download16_1)
desp2 = pd.read_excel(path_download16_2)
desp3 = pd.read_excel(path_download16_3)
desp4 = pd.read_excel(path_download16_4)
desp5 = pd.read_excel(path_download16_5)
desp6 = pd.read_excel(path_download16_6)
desp7 = pd.read_excel(path_download16_7)
desp8 = pd.read_excel(path_download16_8)
desp9 = pd.read_excel(path_download16_9)
desp10 = pd.read_excel(path_download16_10)
desp = pd.concat([desp,desp1,desp2,desp3,desp4,desp5,desp6,desp7,desp8,desp9,desp10],ignore_index=True)
desp['Ano'] = desp.Data.apply(lambda x: x.split('/')[2])
desp['Ano'] = desp['Ano'].apply(str)
desp['Mês'] = desp.Data.apply(lambda x: int(x.split('/')[1]))
desp['Mês'] = desp['Mês'].apply(descobre_mes)
desp['ipca ref'] = desp['Mês']+ ' '+desp['Ano']
desp.drop('Mês',axis=1,inplace=True)
desp.set_index('ipca ref',inplace=True)
desp = desp.join(ipca_full)
desp_temp=desp
result=aplica_inflacao_cam(desp_temp)
result.to_excel(path_download17,index=True)

In [4]:
desp_21=desp[desp['Ano']=='2021']
desp_21

,Empenho,Tipo Empenho,Data,Favorecido,CPF / CNPJ,Licitação,Tipo Licitação,Contrato,Empenhado R$,Liquidado R$,Pago R$,Ano,base = ultimo periodo
fevereiro 2021,'223/2021,ORDINARIO,11/02/2021,FOLHA PESSOAL (EFETIVOS),NaN,',DISPENSADA,',25156.86,0.00,0.00,2021,NaN
fevereiro 2021,'224/2021,ORDINARIO,11/02/2021,FOLHA PESSOAL (EFETIVOS),NaN,',DISPENSADA,',11041.98,0.00,0.00,2021,NaN
fevereiro 2021,'217/2021,ORDINARIO,10/02/2021,SANTA CLARA COMÉRCIO DE UTILIDADES LTDA. - ME,17.325.205/0001-40,'44/2021,DISPENSA,',367.00,0.00,0.00,2021,NaN
fevereiro 2021,'218/2021,ORDINARIO,10/02/2021,FOLHA PESSOAL (COMISSIONADOS),NaN,',DISPENSADA,',264.18,0.00,0.00,2021,NaN
fevereiro 2021,'219/2021,ORDINARIO,10/02/2021,FOLHA PESSOAL (COMISSIONADOS),NaN,',DISPENSADA,',626.30,0.00,0.00,2021,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
janeiro 2021,'14/2021,GLOBAL,01/01/2021,CLARITY SISTEMAS DE ENERGIA LTDA - EPP,03.328.305/0001-15,'47/2019,PREGÃO PRESENCIAL,'50/2019,26219.60,2383.60,0.00,2021,100.00
janeiro 2021,'15/2021,ORDINARIO,01/01/2021,ELEVADORES ATLAS SCHINDLER S/A,00.028.986/0001-08,'3/2016,INEXIGIBILIDADE,',29205.08,2433.67,0.00,2021,100.00
janeiro 2021,'16/2021,GLOBAL,01/01/2021,DANRESA COMÉRCIO DE COMPUTADORES LTDA. - ME,05.550.142/0001-46,'45/2019,PREGÃO PRESENCIAL,'5/2020,127830.17,0.00,0.00,2021,100.00
janeiro 2021,'17/2021,ORDINARIO,01/01/2021,DIHEL COMÉRCIO E PRESTAÇÃO DE SERVIÇOS LTDA - EPP,04.162.919/0001-32,'23/2019,PREGÃO PRESENCIAL,',27499.25,600.00,600.00,2021,100.00


In [203]:
top100 = result.groupby('Favorecido').sum().sort_values('Pago R$',ascending=False).head(100)
top100.drop('base = ultimo periodo',axis=1,inplace=True)
top100['Pago R$']=top100['Pago R$'].apply(lambda x: "{:,.2f}".format(x))
top100['Liquidado R$']=top100['Liquidado R$'].apply(lambda x: "{:,.2f}".format(x))
top100['Empenhado R$']=top100['Empenhado R$'].apply(lambda x: "{:,.2f}".format(x))
top100.to_excel(path_download18,index=True)